<a href="https://colab.research.google.com/github/ABHAY1937/NLP/blob/main/Bidirectional_LSSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake news Classification using Bidirectional LSTM

In [162]:
import pandas as pd
df = pd.read_csv("/content/fack news dataset.csv", engine = 'python', error_bad_lines=False)

<ipython-input-162-812481e21227>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/fack news dataset.csv", engine = 'python', error_bad_lines=False)
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 12
Skippi

In [163]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [164]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [165]:
df.shape

(20822, 5)

In [166]:
df=df.dropna()

In [167]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [168]:
#get the indipendent features
x = df.drop('label',axis=1)

In [169]:
y =df['label']

In [170]:
#check whether the dataset is balanced or not
y.value_counts()

0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: label, dtype: int64

In [171]:
import tensorflow as tf

In [172]:
tf.__version__

'2.12.0'

In [173]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [174]:
## vocabulary size
voc_size = 5000

In [175]:
message = x.copy()

In [176]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [177]:
import nltk

In [178]:
import re
from nltk.corpus import stopwords

In [179]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [180]:
message.reset_index(inplace=True)

In [181]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [182]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [183]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[4041, 2871, 3668, 2061, 479, 2014, 4012, 468, 2027, 4276], [2875, 681, 1331, 3095, 3142, 1505, 4491], [3142, 1753, 1491, 2886], [1046, 1168, 4773, 4937, 4622, 1883], [1852, 3142, 3362, 4593, 2006, 4854, 3142, 2535, 3326, 991], [1548, 509, 3136, 4314, 2118, 2637, 1216, 4231, 2236, 2892, 542, 2197, 4053, 1937, 4491], [4966, 764, 4903, 1045, 2978, 2263, 902, 532, 4095, 504, 846], [2161, 3327, 4081, 4181, 4675, 4168, 2637, 624, 4095, 504, 846], [2211, 1496, 104, 2942, 3010, 68, 138, 1119, 2637, 2115], [182, 3106, 3767, 1455, 1720, 2029, 2920, 371], [4661, 2359, 1075, 669, 1291, 737, 2747, 972, 2040, 4619, 569], [4937, 1916, 479, 68, 2637, 4675], [3041, 2116, 273, 989, 2401, 4659, 4900, 3073, 1490], [4919, 3663, 1996, 1716, 2685, 50, 3460, 4095, 504, 846], [770, 2417, 1576, 2771, 2140, 4095, 504, 846], [2503, 3670, 405, 4081, 3706, 1686, 1542, 3842, 4439, 2892], [4133, 4500, 681], [4128, 583, 595, 902, 2637, 3663, 4734, 4491], [2875, 3309, 1331, 2729, 1578, 1493, 2470, 4583, 4854], [1948,

Embedding Representation


In [184]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ...,  468, 2027, 4276],
       [   0,    0,    0, ..., 3142, 1505, 4491],
       [   0,    0,    0, ..., 1753, 1491, 2886],
       ...,
       [   0,    0,    0, ..., 4095,  504,  846],
       [   0,    0,    0, ..., 3109, 3186, 2186],
       [   0,    0,    0, ..., 3598, 1764, 4552]], dtype=int32)

In [185]:
#Creating model
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [186]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [187]:
## train test split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [188]:
## Model Training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10


UnimplementedError: ignored

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

In [ ]:
print(accuracy_score(y_test,y_pred))
